# TypeScript コーディング問題の解答

## 1. 問題の分析

### 競技プログラミング視点での分析
- **実行速度優先**: 再帰呼び出しのオーバーヘッドを最小化し、配列操作を効率化
- **メモリ使用量**: 結果配列の事前確保は困難なため、動的に構築。コールスタックの深さはO(depth)
- **制約**: 最大深度1000、要素数最大10^5 → スタックオーバーフローのリスクは低い

### 業務開発視点での分析
- **型安全性**: TypeScriptの型システムを活用し、再帰型定義で配列構造を表現
- **可読性**: 再帰的なアプローチが問題の性質に合致し、理解しやすい
- **エラーハンドリング**: 入力検証とエッジケース処理（空配列、n=0など）

### TypeScript特有の考慮点
- **再帰型定義**: `MultiDimensionalArray` は自己参照型で型安全性を確保
- **型ガード**: `Array.isArray()` による実行時型チェック
- **イミュータブル性**: 元の配列を変更せず新しい配列を返す

## 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考 |
|---------|----------|-----------|------------|---------|-------|-----|
| 再帰的展開 | O(N) | O(N + D) | 低 | 高 | 高 | N=全要素数、D=深さ。最も直感的 |
| スタック反復 | O(N) | O(N + D) | 中 | 高 | 中 | スタックオーバーフロー回避可能 |
| reduce連鎖 | O(N²) | O(N + D) | 中 | 中 | 中 | 関数型スタイル、やや複雑 |

## 3. 選択したアルゴリズムと理由

### 選択したアプローチ
**再帰的展開アプローチ**

### 理由
- **計算量的優位性**: 全アプローチがO(N)で同等だが、再帰は最もシンプル
- **TypeScript環境での型安全性**: 再帰型定義との相性が良く、型推論が効果的に働く
- **保守性・可読性**: 問題の再帰的性質を直接コードに反映でき、理解しやすい

### TypeScript特有の最適化ポイント
- **型ガードの活用**: `Array.isArray()` で実行時に安全に型を絞り込み
- **const変数**: イミュータブルな操作を保証
- **スプレッド構文**: 配列のコピーと結合を型安全に実行

## 4. 実装コード

```typescript
// Analyze Complexity
// Runtime 156 ms
// Beats 38.84%
// Memory 90.72 MB
// Beats 60.70%

type MultiDimensionalArray = (number | MultiDimensionalArray)[];

/**
 * 多次元配列を指定された深さまで平坦化する
 * @param arr - 平坦化する多次元配列
 * @param n - 平坦化する深さ（0の場合は平坦化しない）
 * @returns 平坦化された配列
 * @complexity Time: O(N), Space: O(N + D) - N:全要素数, D:深さ
 */
var flat = function (arr: MultiDimensionalArray, n: number): MultiDimensionalArray {
    // n = 0 の場合は平坦化不要
    if (n === 0) {
        return arr;
    }
    
    const result: MultiDimensionalArray = [];
    
    for (const item of arr) {
        // 配列かつ深さがまだ残っている場合は再帰的に平坦化
        if (Array.isArray(item) && n > 0) {
            // 再帰呼び出しで1レベル深く（深さを1減らす）
            const flattened = flat(item, n - 1);
            // スプレッド構文で結果配列に展開
            result.push(...flattened);
        } else {
            // 配列でないか、深さ制限に達した場合はそのまま追加
            result.push(item);
        }
    }
    
    return result;
};
```

### 代替実装：メモリ最適化版（事前サイズ計算なし）

```typescript
// Time Limit Exceeded
// 127 / 131 testcases passed

/**
 * reduceを使った関数型スタイルの実装
 * @complexity Time: O(N), Space: O(N + D)
 */
var flatReduce = function (arr: MultiDimensionalArray, n: number): MultiDimensionalArray {
    if (n === 0) return arr;
    
    return arr.reduce<MultiDimensionalArray>((acc, item) => {
        if (Array.isArray(item) && n > 0) {
            return acc.concat(flatReduce(item, n - 1));
        }
        return acc.concat(item);
    }, []);
};
```

### 代替実装：スタック反復版（スタックオーバーフロー回避）

```typescript
// Analyze Complexity
// Runtime 100 ms
// Beats 73.26%
// Memory 81.66 MB
// Beats 75.35%

/**
 * 反復的なスタックベース実装（大規模データ対応）
 * @complexity Time: O(N), Space: O(N + D)
 */
var flatIterative = function (arr: MultiDimensionalArray, n: number): MultiDimensionalArray {
    const result: MultiDimensionalArray = [];
    const stack: Array<{ item: number | MultiDimensionalArray; depth: number }> = [];
    
    // 初期化：全要素を深さ0でスタックに積む
    for (let i = arr.length - 1; i >= 0; i--) {
        stack.push({ item: arr[i], depth: 0 });
    }
    
    while (stack.length > 0) {
        const { item, depth } = stack.pop()!;
        
        if (Array.isArray(item) && depth < n) {
            // 配列を逆順でスタックに積む（元の順序を保持）
            for (let i = item.length - 1; i >= 0; i--) {
                stack.push({ item: item[i], depth: depth + 1 });
            }
        } else {
            result.push(item);
        }
    }
    
    return result;
};
```

## 5. TypeScript固有の最適化観点

### 型安全性の活用

1. **再帰型定義による構造の表現**
   - `MultiDimensionalArray` の自己参照により、任意深度の配列を型レベルで表現
   - コンパイル時に不正な型の混入を防止

2. **型ガードによる実行時安全性**
   - `Array.isArray()` で配列判定を行い、TypeScriptが自動的に型を絞り込み
   - `number | MultiDimensionalArray` から `MultiDimensionalArray` への安全な型変換

3. **ジェネリクスの制約**
   - 今回は具体的な型が定義済みだが、拡張時にジェネリクスで柔軟性を確保可能

### 実装パターンの選択

- **再帰版**: 最も可読性が高く、型推論が効果的。制約範囲内で安全
- **reduce版**: 関数型スタイルで副作用なし、やや型注釈が必要
- **反復版**: 極端に深い配列（depth > 10000など）でも安全動作

### パフォーマンス考察

- **再帰呼び出しコスト**: 現代のJSエンジンは末尾再帰最適化を持たないが、制約範囲（depth ≤ 1000）では問題なし
- **配列操作**: `push(...array)` はスプレッド展開により内部で配列を反復・割り当てするため、ホットパスでは明示的なループより遅い場合がある（本実装では `push(...flattened)` を要素ごとの `push` に変更することで156ms→80msに改善）
- **メモリ**: 結果配列は避けられないO(N)。コールスタックはO(D)で十分小さい

## パフォーマンス分析

### 現状の問題点

1. **再帰版（メイン実装）の問題**
   - `result.push(...flattened)` のスプレッド演算子が**大規模配列で非効率**
   - スプレッド展開は内部的に全要素をコピーするため、配列サイズに比例してコストが増加

2. **reduce版の致命的問題**
   - `concat()` が**毎回新しい配列を生成**するため、実質的にO(N²)の動作
   - Time Limit Exceeded は予想通りの結果

3. **スタック反復版が最速の理由**
   - スプレッド演算子やconcat()を使わず、`push()` で1要素ずつ追加
   - 関数呼び出しのオーバーヘッドがない

## 改善版実装

### 最適化された再帰版（推奨）

```typescript
// Analyze Complexity
// Runtime 80 ms
// Beats 84.88%
// Memory 76.02 MB
// Beats 85.12%

type MultiDimensionalArray = (number | MultiDimensionalArray)[];

/**
 * 最適化された再帰的平坦化
 * スプレッド演算子を排除し、ループでpush
 * @complexity Time: O(N), Space: O(N + D)
 */
var flat = function (arr: MultiDimensionalArray, n: number): MultiDimensionalArray {
    const result: MultiDimensionalArray = [];
    
    function flatten(items: MultiDimensionalArray, depth: number): void {
        for (const item of items) {
            if (Array.isArray(item) && depth > 0) {
                // 再帰呼び出し（スプレッド演算子を使わない）
                flatten(item, depth - 1);
            } else {
                // 1要素ずつ追加
                result.push(item);
            }
        }
    }
    
    flatten(arr, n);
    return result;
};
```

**改善ポイント:**
- ✅ スプレッド演算子 `...` を完全排除
- ✅ 内部関数で `result` を共有し、毎回の配列生成を回避
- ✅ ループで1要素ずつ `push()` することで定数時間操作

### さらに最適化されたスタック反復版

```typescript
// Wrong Answer
// 86 / 131 testcases passed

/**
 * 最速のスタック反復実装（改良版）
 * @complexity Time: O(N), Space: O(N + D)
 */
var flat = function (arr: MultiDimensionalArray, n: number): MultiDimensionalArray {
    if (n === 0) return arr;
    
    const result: MultiDimensionalArray = [];
    const stack: Array<[MultiDimensionalArray | number, number]> = [[arr, 0]];
    
    while (stack.length > 0) {
        const [item, depth] = stack.pop()!;
        
        if (Array.isArray(item)) {
            if (depth < n) {
                // 逆順でスタックに積む（元の順序を保持）
                for (let i = item.length - 1; i >= 0; i--) {
                    stack.push([item[i], depth + 1]);
                }
            } else {
                // 深さ制限到達、配列全体を追加
                result.push(item);
            }
        } else {
            result.push(item);
        }
    }
    
    return result;
};
```

**改善ポイント:**
- ✅ タプル型 `[item, depth]` でオブジェクト生成コストを削減
- ✅ 初期スタックを `[[arr, 0]]` として1回の push で初期化
- ✅ 型チェックの回数を最小化

### ハイブリッド実装（浅い配列は再帰、深い配列はスタック）

```typescript
// Wrong Answer
// 123 / 131 testcases passed

/**
 * 深さに応じて最適な手法を選択
 * @complexity Time: O(N), Space: O(N + D)
 */
var flat = function (arr: MultiDimensionalArray, n: number): MultiDimensionalArray {
    // 浅い平坦化は再帰で高速化（関数呼び出しコストが問題にならない）
    if (n <= 3) {
        const result: MultiDimensionalArray = [];
        
        function flatten(items: MultiDimensionalArray, depth: number): void {
            for (const item of items) {
                if (Array.isArray(item) && depth > 0) {
                    flatten(item, depth - 1);
                } else {
                    result.push(item);
                }
            }
        }
        
        flatten(arr, n);
        return result;
    }
    
    // 深い平坦化はスタックで安全性確保
    const result: MultiDimensionalArray = [];
    const stack: Array<[MultiDimensionalArray | number, number]> = [[arr, 0]];
    
    while (stack.length > 0) {
        const [item, depth] = stack.pop()!;
        
        if (Array.isArray(item)) {
            if (depth < n) {
                for (let i = item.length - 1; i >= 0; i--) {
                    stack.push([item[i], depth + 1]);
                }
            } else {
                result.push(item);
            }
        } else {
            result.push(item);
        }
    }
    
    return result;
};
```

## 予想されるパフォーマンス

| 実装 | Runtime予想 | Memory予想 | 特徴 |
|-----|-----------|-----------|------|
| **最適化再帰版** | ~100-120ms (65-75%) | ~82-85MB (70-75%) | シンプルで可読性高、十分高速 |
| **改良スタック版** | ~95-105ms (75-80%) | ~80-83MB (75-80%) | 最速候補、やや複雑 |
| **ハイブリッド版** | ~98-110ms (70-78%) | ~81-84MB (72-77%) | バランス型 |

## 推奨実装

**最適化再帰版**を推奨します：

```typescript
// Analyze Complexity
// Runtime 65 ms
// Beats 98.37%
// Memory 74.89 MB
// Beats 90.70%

var flat = function (arr: MultiDimensionalArray, n: number): MultiDimensionalArray {
    const result: MultiDimensionalArray = [];
    
    function flatten(items: MultiDimensionalArray, depth: number): void {
        for (const item of items) {
            if (Array.isArray(item) && depth > 0) {
                flatten(item, depth - 1);
            } else {
                result.push(item);
            }
        }
    }
    
    flatten(arr, n);
    return result;
};
```

**理由:**
- ✅ スタック反復版に匹敵する速度が期待できる
- ✅ コードが簡潔で保守性が高い
- ✅ TypeScriptの型推論が効果的に機能
- ✅ 制約範囲（depth ≤ 1000）で安全

不正解となった2つの実装の問題点と修正案を提示します。

## 問題の分析

両方とも**スタック初期化の誤り**が原因です。

### 誤りの箇所

```typescript
// ❌ 間違い：配列全体を1つの要素として扱ってしまう
const stack: Array<[MultiDimensionalArray | number, number]> = [[arr, 0]];
```

この初期化では、`arr` 全体が1つの要素として深さ0で処理されます。
例えば `arr = [1, 2, [3]]` の場合、`arr` そのものが深さ0の配列として扱われ、意図しない展開が発生します。

### 正しい初期化

```typescript
// ✅ 正しい：配列の各要素を個別にスタックに積む
const stack: Array<[MultiDimensionalArray | number, number]> = [];
for (let i = arr.length - 1; i >= 0; i--) {
    stack.push([arr[i], 0]);
}
```

## 修正版

### さらに最適化されたスタック反復版（修正版）

```typescript
// Analyze Complexity
// Runtime 98 ms
// Beats 73.49%
// Memory 84.92 MB
// Beats 72.56%

/**
 * 修正されたスタック反復実装
 * @complexity Time: O(N), Space: O(N + D)
 */
var flat = function (arr: MultiDimensionalArray, n: number): MultiDimensionalArray {
    if (n === 0) return arr;
    
    const result: MultiDimensionalArray = [];
    const stack: Array<[MultiDimensionalArray | number, number]> = [];
    
    // 修正：各要素を個別に深さ0でスタックに積む
    for (let i = arr.length - 1; i >= 0; i--) {
        stack.push([arr[i], 0]);
    }
    
    while (stack.length > 0) {
        const [item, depth] = stack.pop()!;
        
        if (Array.isArray(item) && depth < n) {
            // 配列を逆順でスタックに積む
            for (let i = item.length - 1; i >= 0; i--) {
                stack.push([item[i], depth + 1]);
            }
        } else {
            // 数値または深さ制限到達の配列
            result.push(item);
        }
    }
    
    return result;
};
```

### ハイブリッド実装（修正版）

```typescript
// Analyze Complexity
// Runtime 91 ms
// Beats 76.98%
// Memory 84.08 MB
// Beats 72.56%

/**
 * 深さに応じて最適な手法を選択（修正版）
 * @complexity Time: O(N), Space: O(N + D)
 */
var flat = function (arr: MultiDimensionalArray, n: number): MultiDimensionalArray {
    // 浅い平坦化は再帰で高速化
    if (n <= 3) {
        const result: MultiDimensionalArray = [];
        
        function flatten(items: MultiDimensionalArray, depth: number): void {
            for (const item of items) {
                if (Array.isArray(item) && depth > 0) {
                    flatten(item, depth - 1);
                } else {
                    result.push(item);
                }
            }
        }
        
        flatten(arr, n);
        return result;
    }
    
    // 深い平坦化はスタックで安全性確保
    const result: MultiDimensionalArray = [];
    const stack: Array<[MultiDimensionalArray | number, number]> = [];
    
    // 修正：各要素を個別に深さ0でスタックに積む
    for (let i = arr.length - 1; i >= 0; i--) {
        stack.push([arr[i], 0]);
    }
    
    while (stack.length > 0) {
        const [item, depth] = stack.pop()!;
        
        if (Array.isArray(item) && depth < n) {
            for (let i = item.length - 1; i >= 0; i--) {
                stack.push([item[i], depth + 1]);
            }
        } else {
            result.push(item);
        }
    }
    
    return result;
};
```

## 修正内容のまとめ

| 箇所 | 修正前 | 修正後 |
|-----|-------|-------|
| スタック初期化 | `[[arr, 0]]` | `arr` の各要素を個別に `[arr[i], 0]` として追加 |
| depth判定 | `if (depth < n)` のみ | `if (Array.isArray(item) && depth < n)` |
